In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import scrapbook as sb 
import pandas as pd
import numpy as np 
import seaborn as sns 
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt

# Baseline 

In [3]:
books = sb.read_notebooks("./BaseLine_Model_Output")
baseLine_data = []
for nb in books.notebooks: 
    nbList=[nb.scraps['Stats Model MAE'].data,nb.scraps['Catboost MAE'].data]
    baseLine_data.append(nbList)
df = pd.DataFrame(baseLine_data, columns = ["Stats Model","Catboost"])
#baseLine_data = np.reshape(baseLine_data,(2,10))
display(df)
print(df.mean())

,Stats Model,Catboost
0,0.384634,0.143490
1,0.679533,0.249754
2,0.658534,0.723003
3,0.575827,0.536705
4,0.521046,0.334735
5,0.560545,0.214166
6,0.394293,0.354267
7,0.558400,0.228594
8,0.543642,0.486789
9,0.624801,0.789319


Stats Model    0.550126
Catboost       0.406082
dtype: float64


# GAN Model

In [4]:
book = sb.read_notebooks("./GAN_Output")
gan_data = []
gan_mse = []
for nb in book.notebooks:
    metrics = nb.scraps['GAN_1 Metrics'].data
    for i in range(1000):
        gan_mse.append(metrics[0][i])
    nbList = [nb.scraps['GAN Model MSE'].data,
              nb.scraps['GAN Model MAE'].data,
              nb.scraps['GAN Model Euclidean distance'].data,
              nb.scraps['GAN Model Manhattan Distance'].data]        
    gan_data.append(nbList)
    
df = pd.DataFrame(gan_data, columns = ['MSE','MAE','Euclidean Distance','Manhattan Distance'])
display(df.style)
print("MEAN:")
print(df.mean(axis = 0))
gan_data = np.array(gan_data)

,MSE,MAE,Euclidean Distance,Manhattan Distance
0,0.155729,0.243533,1.764267,4.870652
1,0.480955,0.581771,3.101088,11.635422
2,0.329641,0.404866,2.567177,8.097312
3,0.411285,0.566355,2.867202,11.327104
4,0.252376,0.390330,2.246448,7.806599
5,0.205691,0.367313,2.027688,7.346263
6,0.106401,0.253454,1.458104,5.069089
7,0.268535,0.420745,2.317349,8.414902
8,0.686369,0.581106,3.704330,11.622122
9,0.211097,0.358662,2.054277,7.173240


MEAN:
MSE                   0.310808
MAE                   0.416814
Euclidean Distance    2.410793
Manhattan Distance    8.336271
dtype: float64


# ABC_GAN Analysis

## ABC Pre-generator - Catboost 


In [5]:
book = sb.read_notebooks("./ABC_GAN_Catboost")
paramVal = [1,0.1,0.01]
abc_mae = [[] for i in range(3)]
abc_mae_skip = [[] for i in range(3)]
abc_mae_mean = [[] for i in range(3)]
abc_mae_skip_mean = [[] for i in range(3)]
abc_weights = [[] for i in range(3)]
prior_model = [[] for i in range(3)]
abc_pre_generator = [[] for i in range(3)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    
    #Divide data according to parameters 
    for i in range(3):
        if paramVar == paramVal[i]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [6]:
for i in range(3):
    data = []
    for j in range(len(abc_weights[i])):
        data.append([paramVal[i],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data, columns = ['Variance','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.mean(axis=0))
    print("-------------------------------------------------------------")

,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.42621,0.82744,0.54711,0.54191,0.53726
1,1,0.65297,0.99704,0.46737,0.51650,0.50965
2,1,0.38057,0.88378,0.86772,0.40616,0.40200
3,1,0.45171,0.94320,0.18296,0.59997,0.49786
4,1,0.39427,1.25127,0.15235,0.44808,0.48046
5,1,0.34438,0.88897,0.26114,0.39261,0.39230
6,1,0.32108,1.01084,0.22016,0.28764,0.35946
7,1,0.35707,1.26602,0.46595,0.25860,0.31355
8,1,0.51546,0.94426,0.59526,0.47219,0.44014
9,1,0.44264,1.21359,0.86627,0.58922,0.60535


Variance                         1.000000
Prior Model MAE                  0.428637
ABC pre-generator MAE            1.022641
Skip Node weight                 0.462629
ABC GAN MAE                      0.451289
ABC_GAN MAE (skip connection)    0.453804
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.53668,0.51651,0.62019,0.54104,0.51962
1,0.1,0.47355,0.47965,0.53377,0.46734,0.46421
2,0.1,0.33190,0.38638,0.29185,0.34245,0.31963
3,0.1,0.39706,0.41030,0.43129,0.37943,0.40711
4,0.1,0.34925,0.32784,0.31799,0.36975,0.36937
5,0.1,0.51984,0.51396,0.34955,0.51153,0.52991
6,0.1,0.35432,0.35693,0.10565,0.30971,0.36567
7,0.1,0.53064,0.56628,0.02329,0.55765,0.55152
8,0.1,0.71232,0.71499,0.13962,0.71460,0.76575
9,0.1,0.36771,0.39040,0.64944,0.30918,0.32917


Variance                         0.100000
Prior Model MAE                  0.457329
ABC pre-generator MAE            0.466326
Skip Node weight                 0.346264
ABC GAN MAE                      0.450268
ABC_GAN MAE (skip connection)    0.462195
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.33287,0.33551,0.00000,0.33310,0.33333
1,0.01,0.42998,0.42820,0.00000,0.42432,0.43007
2,0.01,0.50122,0.50413,0.06712,0.53061,0.51043
3,0.01,0.43676,0.44261,0.00000,0.45985,0.43671
4,0.01,0.33455,0.33644,0.07439,0.35703,0.34343
5,0.01,0.78088,0.77916,0.23687,0.81768,0.80321
6,0.01,0.47672,0.47625,0.65262,0.46068,0.50837
7,0.01,0.38560,0.38509,0.06908,0.41339,0.39242
8,0.01,0.39985,0.40152,0.66324,0.39199,0.41233
9,0.01,0.48295,0.48434,0.25610,0.48473,0.48003


Variance                         0.010000
Prior Model MAE                  0.456137
ABC pre-generator MAE            0.457325
Skip Node weight                 0.201942
ABC GAN MAE                      0.467338
ABC_GAN MAE (skip connection)    0.465033
dtype: float64
-------------------------------------------------------------


## ABC Pre-generator - Stats 


In [7]:
book = sb.read_notebooks("./ABC_GAN_Stats")
paramVal = [1,0.1,0.01]
abc_mae = [[] for i in range(3)]
abc_mae_skip = [[] for i in range(3)]
abc_mae_mean = [[] for i in range(3)]
abc_mae_skip_mean = [[] for i in range(3)]
abc_weights = [[] for i in range(3)]
prior_model = [[] for i in range(3)]
abc_pre_generator = [[] for i in range(3)]

for nb in book.notebooks:
    metrics1 = np.array(nb.scraps['ABC_GAN_1 Metrics'].data)
    metrics3 = np.array(nb.scraps['ABC_GAN_3 Metrics'].data)
    paramVar = float(nb.papermill_dataframe.iloc[0]['value'])
    
    #Divide data according to parameters 
    for i in range(3):
        if paramVar == paramVal[i]:
            for j in range(100):
                abc_mae[i].append(metrics1[1,j])
                abc_mae_skip[i].append(metrics3[1,j])
            abc_weights[i].append(nb.scraps['Skip Connection Weight'].data)
            prior_model[i].append(nb.scraps['Prior Model MSE'].data)
            abc_pre_generator[i].append(nb.scraps['ABC Pre-generator MSE'].data)
            abc_mae_skip_mean[i].append(mean(metrics3[1,:]))
            abc_mae_mean[i].append(mean(metrics1[1,:]))

In [8]:
for i in range(3):
    data = []
    for j in range(len(abc_weights[i])):
        data.append([paramVal[i],prior_model[i][j],
                     abc_pre_generator[i][j],abc_weights[i][j],abc_mae_mean[i][j],abc_mae_skip_mean[i][j]])
    
    df = pd.DataFrame(data, columns = ['Variance','Prior Model MAE',
                                       'ABC pre-generator MAE','Skip Node weight','ABC GAN MAE','ABC_GAN MAE (skip connection)'])
    display(df.round(5))
    print(df.mean(axis=0))
    print("-------------------------------------------------------------")

,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,1,0.56200,0.95637,0.98965,0.58176,0.46248
1,1,0.59398,0.99105,1.00000,0.45958,0.42152
2,1,0.57489,1.09791,0.99835,0.41223,0.33756
3,1,0.53514,1.14123,0.98582,0.41946,0.54810
4,1,0.49943,1.00669,0.99685,0.41806,0.38833
5,1,0.40170,0.92629,0.98808,0.39980,0.42166
6,1,0.40741,0.70083,1.00000,0.41765,0.36945
7,1,0.65974,0.91241,0.99395,0.38540,0.33890
8,1,0.51646,0.62034,0.98755,0.59699,0.65632
9,1,0.71367,1.12624,0.98142,0.77017,0.60436


Variance                         1.000000
Prior Model MAE                  0.546443
ABC pre-generator MAE            0.947936
Skip Node weight                 0.992167
ABC GAN MAE                      0.486111
ABC_GAN MAE (skip connection)    0.454868
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.1,0.63862,0.63696,0.56790,0.40297,0.43240
1,0.1,0.61852,0.60149,0.35249,0.58207,0.48299
2,0.1,0.54850,0.53720,0.64919,0.63439,0.51157
3,0.1,0.47923,0.49727,0.73570,0.57385,0.40636
4,0.1,0.57902,0.61939,0.86923,0.43976,0.22853
5,0.1,0.42007,0.44082,0.44208,0.40088,0.34960
6,0.1,0.63264,0.65136,0.35998,0.56938,0.48246
7,0.1,0.35147,0.37293,0.74450,0.44832,0.30263
8,0.1,0.42532,0.45043,0.41286,0.32823,0.43937
9,0.1,0.58336,0.56493,0.31535,0.67975,0.63128


Variance                         0.100000
Prior Model MAE                  0.527676
ABC pre-generator MAE            0.537278
Skip Node weight                 0.544928
ABC GAN MAE                      0.505961
ABC_GAN MAE (skip connection)    0.426718
dtype: float64
-------------------------------------------------------------


,Variance,Prior Model MAE,ABC pre-generator MAE,Skip Node weight,ABC GAN MAE,ABC_GAN MAE (skip connection)
0,0.01,0.50899,0.51005,0.17819,0.60887,0.48184
1,0.01,0.59421,0.59443,0.28878,0.63566,0.46730
2,0.01,0.61136,0.60947,0.25617,0.57071,0.55873
3,0.01,0.59228,0.59332,0.24635,0.51744,0.47719
4,0.01,0.51754,0.51260,0.18660,0.60983,0.52483
5,0.01,0.52575,0.52621,0.31640,0.41617,0.46530
6,0.01,0.47344,0.47543,0.20956,0.29510,0.35082
7,0.01,0.53252,0.52901,0.31251,0.38626,0.37939
8,0.01,0.58162,0.58059,0.17613,0.77086,0.69248
9,0.01,0.49243,0.49508,0.16200,0.43190,0.53707


Variance                         0.010000
Prior Model MAE                  0.543013
ABC pre-generator MAE            0.542619
Skip Node weight                 0.233269
ABC GAN MAE                      0.524280
ABC_GAN MAE (skip connection)    0.493495
dtype: float64
-------------------------------------------------------------
